In [2]:
from pyspark.sql import SparkSession

In [17]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
config = SparkConf()
spark = SparkSession.builder.master("local").appName("test").config(conf=config).getOrCreate()

Exception: Java gateway process exited before sending its port number

In [15]:
df = sc.read.option("header", "true").option("inferSchema", "true").csv("data\train.csv") 
df.printSchema()

AttributeError: 'Builder' object has no attribute 'read'